In [ ]:
!tensorboard --logdir="../pytorch-transformer_2/translation_trial/zulu_english_segmenter_two/tensor_data" --bind_all --port 6006

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.18.0 at http://b378ea036eb9:6006/ (Press CTRL+C to quit)


In [2]:
pip install fasttext

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=325277 sha256=fdd7b339ff58ecf4ecef5c066af2a2977d73ffe3ca737e67032348ccbdc2a777
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [31]:
import json
import os
import fasttext
import re
import csv

model = fasttext.load_model('lid201-model.bin')
lang_prediction = model.predict("ke a mo rata")
lang_label = lang_prediction[0][0].split("__label__")[1]
lang_label

'tsn_Latn'

In [26]:
import json
import os
import fasttext
import re
import csv

model = fasttext.load_model('lid201-model.bin')

def is_zulu_xhosa_swati(line):
    lang_prediction = model.predict(line)
    lang_label = lang_prediction[0][0].split("__label__")[1]
    if lang_label in ['zul_Latn']:
        return True
    return False

def is_english(line):
    lang_prediction = model.predict(line)
    lang_label = lang_prediction[0][0].split("__label__")[1]
    if lang_label in ['eng_Latn']:
        return True
    return False

def filter_parallel_text(input_file, accepted_file, rejected_file, model, target_accepted=100000):
    accepted_pairs = []
    rejected_pairs = []
    total_processed = 0
    unique_zulu = set()  # Track unique Zulu sentences
    
    while len(accepted_pairs) < target_accepted:
        with open(input_file, 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            header = next(reader)  # Skip header
            
            for row in enumerate(reader):
                # Skip rows we've already processed in previous iterations
                if row[0] < total_processed:
                    continue
                    
                if len(row[1]) != 2:
                    continue
                    
                eng_text, zulu_text = row[1]
                
                # Check if both texts pass their respective language tests
                valid_eng = is_english(eng_text)
                valid_zulu = is_zulu_xhosa_swati(zulu_text)
                is_unique_zulu = zulu_text not in unique_zulu
                
                # Store the pair and the reason for rejection if any
                if valid_eng and valid_zulu and is_unique_zulu:
                    accepted_pairs.append((eng_text, zulu_text))
                    unique_zulu.add(zulu_text)  # Add to set of unique Zulu sentences
                    if len(accepted_pairs) >= target_accepted:
                        break
                else:
                    reason = []
                    if not valid_eng:
                        reason.append("Failed English check")
                    if not valid_zulu:
                        reason.append("Failed Zulu/Xhosa/Swati check")
                    if not is_unique_zulu:
                        reason.append("Duplicate Zulu sentence")
                    rejected_pairs.append({
                        "english": eng_text,
                        "zulu": zulu_text,
                        "reason": " & ".join(reason)
                    })
                
                total_processed += 1
                
                # Print progress periodically
                if total_processed % 1000 == 0:
                    print(f"Processed: {total_processed}, Accepted: {len(accepted_pairs)}, Unique Zulu: {len(unique_zulu)}")
        
        # If we've processed all rows but still haven't found enough accepted pairs
        if total_processed == sum(1 for row in csv.reader(open(input_file))) - 1:
            print("Reached end of file before finding enough accepted pairs.")
            break
    
    # Write accepted pairs to CSV
    with open(accepted_file, 'w', encoding='utf-8', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['english', 'zulu'])
        writer.writerows(accepted_pairs)
    
    # Write rejected pairs to JSON for better readability of rejection reasons
    with open(rejected_file, 'w', encoding='utf-8') as outfile:
        json.dump(rejected_pairs, outfile, indent=2, ensure_ascii=False)
    
    return accepted_pairs, rejected_pairs

# Usage
input_file = 'eng_zul_nllb_data.csv'
accepted_file = 'accepted_pairs_.csv'
rejected_file = 'rejected_pairs.json'

accepted, rejected = filter_parallel_text(input_file, accepted_file, rejected_file, model)
print(f"\nFinal Results:")
print(f"Total pairs processed: {len(accepted) + len(rejected)}")
print(f"Accepted pairs: {len(accepted)}")
print(f"Rejected pairs: {len(rejected)}")
print(f"\nResults saved to:")
print(f"- Accepted pairs: {accepted_file}")
print(f"- Rejected pairs: {rejected_file}")

Processed: 1000, Accepted: 863, Unique Zulu: 863
Processed: 2000, Accepted: 1740, Unique Zulu: 1740
Processed: 3000, Accepted: 2631, Unique Zulu: 2631
Processed: 4000, Accepted: 3507, Unique Zulu: 3507
Processed: 5000, Accepted: 4408, Unique Zulu: 4408
Processed: 6000, Accepted: 5310, Unique Zulu: 5310
Processed: 7000, Accepted: 6204, Unique Zulu: 6204
Processed: 8000, Accepted: 7109, Unique Zulu: 7109
Processed: 9000, Accepted: 8003, Unique Zulu: 8003
Processed: 10000, Accepted: 8889, Unique Zulu: 8889
Processed: 11000, Accepted: 9773, Unique Zulu: 9773
Processed: 12000, Accepted: 10664, Unique Zulu: 10664
Processed: 13000, Accepted: 11561, Unique Zulu: 11561
Processed: 14000, Accepted: 12456, Unique Zulu: 12456
Processed: 15000, Accepted: 13356, Unique Zulu: 13356
Processed: 16000, Accepted: 14265, Unique Zulu: 14265
Processed: 17000, Accepted: 15140, Unique Zulu: 15140
Processed: 18000, Accepted: 16027, Unique Zulu: 16027
Processed: 19000, Accepted: 16910, Unique Zulu: 16910
Process

In [51]:
import re

def preprocess_text(text):
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

with open("eng_Latn.dev", "r") as file:
    sentences = file.readlines()

cleaned_sentences = [preprocess_text(sentence.strip()) for sentence in sentences]

with open("flores_english_test_val.dev", "w") as file:
    for sentence in cleaned_sentences:
        file.write(sentence + "\n")

print("Preprocessing complete. Cleaned text saved to 'flores_english_test_val.dev")

Preprocessing complete. Cleaned text saved to 'flores_english_test_val.dev


In [9]:
df3 = pd.read_csv('./embedding_segmenter_three.csv')
df3

,original,segmenter_three
0,ab ebgngilangazelela ukusondela\n,ab ebgngi-langazelel-a uku-sondel-a
1,aba english ulimi yenza\n,a-b-a engli-sh ulimi yenz-a
2,aba adl naba florinadusayangane\n,a-b-a a-dl nab-a florinadusay-anga-ne
3,aba bantu badala kulindeleke ukuba bayokumela ...,a-b-a bantu ba-dal-a ku-lindelek-e uku-b-a bay...
4,aba imiklamo yakho emkhakheni\n,a-b-a imiklamo ya-kho emkhakheni
...,...,...
499995,zwi zwa ama zwi na khakhathi\n,zw-i zwa ama zw-i n-a khakhathi
499996,zwimela zwo no nga zwikhopha\n,zwimel-a zwo no nga zw-ikhopha
499997,zwisisani amazwi ami\n,zwisis-a-ni amazwi ami
499998,zwothe zwa aquaponics zwine zwine zwa vha zwav...,zwoth-e zwa a-quaponics-a zwin-e zwin-e zwa vh...


In [12]:
import pandas as pd

# Read the first CSV with English and Zulu
df1 = pd.read_csv('./diversified_data_segmenter_one.csv')

# Read the second CSV with original and segmented Zulu
df2 = pd.read_csv('./diversified_data_segmenter_two_02.csv')


# Read the second CSV with original and segmented Zulu
df3 = pd.read_csv('./embedding_segmenter_three.csv')


# Create new dataframe with desired columns
final_df = pd.DataFrame({
    'isizulu': df1['original'],
    'segmenter_one': df1['segmenter_one'],
    'segmenter_two': df2['segmenter_two'],
    'segmenter_two': df3['segmenter_three']
})



# Save to new CSV
final_df.to_csv('word_embedding_data_500.csv', index=False)

# Display first few rows to verify
final_df.head()

,isizulu,segmenter_one,segmenter_two
0,ab ebgngilangazelela ukusondela\n,ab- e-bg-ngi-langazelel-a uku-sondel-a,ab ebgngi-langazelel-a uku-sondel-a
1,aba english ulimi yenza\n,ab-a e-ng-li-sh u-limi y-enz-a,a-b-a engli-sh ulimi yenz-a
2,aba adl naba florinadusayangane\n,ab-a a-dl n-ab-a folorindusay-angane,a-b-a a-dl nab-a florinadusay-anga-ne
3,aba bantu badala kulindeleke ukuba bayokumela ...,ab-a ba-ntu b-a-dal-a ku-lindelek-e uku-b-a ba...,a-b-a bantu ba-dal-a ku-lindelek-e uku-b-a bay...
4,aba imiklamo yakho emkhakheni\n,ab-a i-mi-klamo ya-kho e-mkhakh-eni,a-b-a imiklamo ya-kho emkhakheni


In [11]:
import pandas as pd


data = pd.read_csv("word_embedding_data_500.csv")
# Get a random row
random_row = data.sample(n=1)

# Print the row in a nicely formatted way
print("\nRandom sample row:")
print("-" * 80)
print(f"English: {random_row['english'].values[0]}")
print(f"isiZulu: {random_row['original'].values[0]}")
print(f"Segmented isiZulu: {random_row['segmenter_one'].values[0]}")
print("-" * 80)


Random sample row:
--------------------------------------------------------------------------------


KeyError: 'english'

In [4]:
# Print row at specific index (e.g., index 5)
index = 2005  # Change this to any index you want
row = final_df.iloc[index]

print("\nRow at index", index)
print("-" * 80)
print(f"English: {random_row['english'].values[0]}")
print(f"isiZulu: {random_row['original'].values[0]}")
print(f"Segmented isiZulu: {random_row['segmenter_one'].values[0]}")
print("-" * 80)


Row at index 2005
--------------------------------------------------------------------------------
English: high winds hail excessive precipitation and wildfires are forms and effects of severe weather as are thunderstorms tornadoes waterspouts and cyclones
isiZulu: imimoya ephakeme isangqotho isimo sezulu esidlulele kanye nemililo yasendle ziwuhlobo nomphumela wesimo sezulu esibi kakhulu kanjalo neziphepho izivunguvungu izishomo zamanzi amuncwa yisivunguvungu kanye nezihlwithi

Segmented isiZulu: i-mi-moya e-phakem-e i-s-angqotho i-si-mo se-zulu esi-dlulel-e ka-nye n-e-mi-lilo ya-sendle zi-w-u-hlobo n-o-m-phumela we-si-mo se-zulu esi-bi ka-khulu ka-njalo n-e-zi-phepho i-zi-vunguvungu i-zi-shomo za-ma-nzi a-muncw-a y-i-si-vunguvungu ka-nye n-e-zi-hlwithi
--------------------------------------------------------------------------------


In [65]:
import pandas as pd
from datetime import datetime

def combine_csv_and_text(csv_path, text_path, output_path=None, column_names=None, new_column_name='text_content'):
    """
    Combine a CSV file with three columns and a plain text file, with option to rename columns.
    
    Args:
        csv_path (str): Path to the CSV file
        text_path (str): Path to the text file
        output_path (str, optional): Path for output file. If None, generates timestamped name
        column_names (list, optional): New names for the CSV columns
        new_column_name (str, optional): Name for the column from text file
        
    Returns:
        str: Path to the saved output file
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        print(f"\nRead CSV file with shape: {df.shape}")
        print("Original CSV columns:", df.columns.tolist())
        
        # Rename columns if new names provided
        if column_names:
            if len(column_names) != len(df.columns):
                print(f"Warning: Number of provided column names ({len(column_names)}) "
                      f"doesn't match CSV columns ({len(df.columns)})")
                print("Will use provided names for available columns")
            
            # Rename only the columns we have names for
            for i, new_name in enumerate(column_names):
                if i < len(df.columns):
                    df = df.rename(columns={df.columns[i]: new_name})
        
        # Read the text file
        with open(text_path, 'r', encoding='utf-8') as file:
            text_lines = [line.strip() for line in file.readlines() if line.strip()]
        print(f"Read {len(text_lines)} lines from text file")
        
        # Verify lengths match
        if len(text_lines) != len(df):
            print(f"\nWarning: Length mismatch!")
            print(f"CSV rows: {len(df)}")
            print(f"Text lines: {len(text_lines)}")
            shorter_len = min(len(df), len(text_lines))
            print(f"Will use first {shorter_len} entries from each file")
            
            # Truncate to shorter length
            df = df.iloc[:shorter_len]
            text_lines = text_lines[:shorter_len]
        
        # Add text file content as new column
        df[new_column_name] = text_lines
        
        # Generate output path if not provided
        if output_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f'combined_output_{timestamp}.csv'
        
        # Save combined data
        df.to_csv(output_path, index=False)
        
        # Print sample of combined data
        print("\nFirst few rows of combined data:")
        print(df.head().to_string())
        
        # Print statistics
        print(f"\nCombined data statistics:")
        print(f"Total rows: {len(df)}")
        print(f"Total columns: {len(df.columns)}")
        print("Final columns:", df.columns.tolist())
        print(f"\nOutput saved to: {output_path}")
        
        return output_path
        
    except FileNotFoundError as e:
        print(f"Error: File not found - {e.filename}")
    except pd.errors.EmptyDataError:
        print(f"Error: The CSV file is empty")
    except Exception as e:
        print(f"Error: An unexpected error occurred - {str(e)}")

# Example usage:
if __name__ == "__main__":
    # Example with column renaming
    csv_file = "flores_test_val.csv"  # CSV with three columns
    text_file = "flores_english_test_val.dev" # Plain text file
    
    # Specify new column names
    new_column_names = ['isizulu', 'segmenter_one', 'segmenter_two', 'segmenter_three']
    
    # Combine files with new column names
    combined_file = combine_csv_and_text(
        csv_file, 
        text_file,
        column_names=new_column_names,
        new_column_name='english'  # Name for the text file column
    )


Read CSV file with shape: (2009, 4)
Original CSV columns: ['original', 'segmenter_one', 'segmenter_two', 'segmenter_three']
Read 2009 lines from text file

First few rows of combined data:
                                                                                                                                                                                                                                                                                                    isizulu                                                                                                                                                                                                                                                                                                                                                           segmenter_one                                                                                                                                                       